In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('netflix_titles.csv', encoding='latin1')

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.drop(df.filter(regex='^Unnamed'),axis=1,inplace=True)


In [ ]:
df.isna().sum()

In [ ]:
df['duration'].mode()

In [ ]:
df['director'].fillna('Unknown', inplace=True)
df['cast'].fillna('Unknown', inplace=True)
df['country'].fillna('Unknown', inplace=True)
df['rating'].fillna('Unknown',inplace=True)
df['duration'].fillna(df['duration'].mode(),inplace=True)
df['date_added'].fillna('1900-01-01', inplace=True)


In [ ]:
df.isna().sum()

In [ ]:
df[df['duration'].isna()]

In [ ]:
df['type'].value_counts()

In [ ]:
# Plotting Genres on Pie Chart
fig,ax=plt.subplots(figsize=(5,10))
wedges=ax.pie(df['type'].value_counts(),labels=set(df['type'].values),autopct='%1.1f%%')


In [ ]:
# Who are the most popular movie director
movies = df[(df['type']) == 'Movie']
movies = movies[movies.country != 'Unknown']
df['director']=df['director'].fillna('Unknown')
directors=df[df['director']!="Unknown"]['director']
topDir=directors.value_counts().nlargest(10).sort_values(ascending=False)
topDir.index
fig,ax=plt.subplots(1,2,figsize=(15,8))
ax[0].set_title("The most popular movie directors")
sns.barplot(data=movies,x=topDir.values,y=topDir.index,ax=ax[0])

In [ ]:
df['year_add'] = df['date_added'].apply(lambda x : x.split(',')[-1])


plt.figure(figsize=(10,7))
plt.title('Release year on Netflix distribution', fontweight='bold')
plt.xlabel('Release year')
sns.histplot(data=df, x=df['year_add'].sort_values(ascending=False), bins=30, hue='type', palette='rocket')
plt.show()

In [ ]:
# Convert date_added to datetimedf['date_added'] = pd.to_datetime(df['date_added'])

# Extract year from date_added
df['year_added'] = df['date_added'].dt.year

# Group by year and type
content_trend = df.groupby(['year_added', 'type']).size().reset_index(name="count")

# Setting up two plots
fig, ax = plt.subplots(1, 2, figsize=(15, 8))

# Line plot
sns.lineplot(data=content_trend, x='year_added', y='count', hue='type', ax=ax[0])
ax[0].set_title('Content Added Over Time')
ax[0].set_xlabel('Year')
ax[0].set_ylabel('Count')

# Bar plot
sns.barplot(data=content_trend, x='year_added', y='count', hue='type', ax=ax[1])
ax[1].set_title('Content Added Over Time')
ax[1].set_xlabel('Year')
ax[1].set_ylabel('Count')

plt.show()


In [ ]:
dfGenre=df['listed_in'].str.split(",").explode().str.strip()
dfGenre.value_counts().nlargest(10)
dfGenre=pd.DataFrame(dfGenre.value_counts().nlargest(10))
dfGenre.rename(columns={'listed_in':'count'},inplace=True)
dfGenre.reset_index(names="genre",inplace=True)
fig=plt.figure()
axes=fig.add_axes([0.1,0.1,1,1])
# sns.barplot(x='count', y='genre', data=dfGenre, ax=axes)
sns.barplot(x='genre', y='count', data=dfGenre, ax=axes)
axes.set_title('Genre Count')
axes.set_xlabel('Genre')
axes.set_ylabel('Count')
axes.set_xticklabels(axes.get_xticklabels(), rotation=45, horizontalalignment='right')
# plt.tight_layout()

In [ ]:
# Question: Create a heatmap showing the number of movies and TV shows produced by the top 10 countries.
# Count the number of titles by country and type
country_content = df.groupby(['country', 'type']).size().unstack().fillna(0)
# Select top 10 countries with the most titles
top_countries = country_content.sum(axis=1).nlargest(10).index
country_content_top = country_content.loc[top_countries]

# Plotting
plt.figure(figsize=(12, 8))
sns.heatmap(country_content_top, annot=True, fmt='g', linewidths=.5)
plt.title('Number of Movies and TV Shows Produced by Top 10 Countries')
plt.xlabel('Type')
plt.ylabel('Country')
plt.show()

country_content_top
# country_content['country'].str.strip()

In [ ]:
# Determine if there is a seasonality pattern in the content addition. For example, are more contents added in a particular quarter of the year
df['date_added'] = pd.to_datetime(df['date_added'])
df['quarter']=df['date_added'].dt.quarter
seasonality_trend=df.groupby(['quarter','type']).size().reset_index(name="count")
seasonality_trend

# Bar plot
plt.figure(figsize=(10, 6))
sns.barplot(data=seasonality_trend, x='quarter', y='count', hue='type')
plt.title('Seasonality Trend by Quarter')
plt.xlabel('Quarter')
plt.ylabel('Counts')
plt.show()


In [ ]:
show_dur=df['duration'].str.extract('(\d+)').astype('float')
average_show_duration = np.mean(movie_dur)
print('The average duration of a Movie on Netflix is '+str(average_show_duration)+ ' minutes.')


In [ ]:
# avg duration of a movie on histplot
MovieDur=df[df['type']=='Movie']['duration'].str.extract('(\d+)').astype('float')
plt.figure(figsize=(15,8))
sns.histplot(data=MovieDur,kde=True,legend=False)
plt.title('Movie duration (minutes)', fontweight='bold')
plt.xlabel('Movie duration')

plt.show()

In [ ]:
# AVG DURATION OF TV SHOW
tvDur=df[df['type']=='TV Show']['duration']
tvDur=tvDur.value_counts().sort_values(ascending=True)
tvDur
plt.figure(figsize=(15,8))
sns.barplot(x=tvDur.index,y=tvDur.values)
plt.title('TV duration (seasons)', fontweight='bold')
plt.xlabel('TV duration')
plt.xticks(rotation=45, ha='right')
# plt.yscale('log') goes in power of 10
plt.ylim(0, 5)  # Zooming in on the first 5 counts

plt.show()
